# 1. Data preprocessing

In [3]:
import pandas as pd
import json
from pathlib import Path
from tqdm import tqdm
import pickle
import torch
from sentence_transformers import SentenceTransformer
import os
import re
import time

class Config:
    DATA_FILE_PATH = './dataset/arxiv-metadata-oai-snapshot.json'
    OUTPUT_RAW_DATA_CACHE = './dataset/arxiv_raw_cache.pkl'
    OUTPUT_PROCESSED_DATA_INTERIM = './dataset/processed_arxiv_interim.pkl'
    OUTPUT_PROCESSED_DATA_PATH = './dataset/processed_arxiv.pkl'
    OUTPUT_EMBEDDING_PATH = './dataset/arxiv_embeddings_fp16.pkl'
    MODEL_NAME = 'sentence-transformers/all-mpnet-base-v2'
    BATCH_SIZE = 32
    SAMPLES_PER_YEAR = 100
    PROFILE_SAMPLE_SIZE = 2000

## 1.1. 날짜 및 슬롯 추출 함수

In [2]:
def extract_year_from_versions(versions):
    if isinstance(versions, list) and len(versions) > 0:
        first_version = versions[0]
        created_date = first_version.get('created', '')
        match = re.search(r'(\d{4})', created_date) 
        if match:
            try:
                year = int(match.group(1))
                if 1990 < year <= 2025: # 유효한 연도 범위로 제한
                    return year
            except ValueError:
                return None
    return None

## 1.2. 5년 단위 time slot 생성

In [13]:
def to_5yr_slot(y):
    if y is None or y < 1991:
        return None
    start = ((y - 1) // 5) * 5 + 1 
    end = start + 4
    return f"{start}-{end}"

## 1.3. 데이터 로드 및 원본 캐시 생성 (Raw Data Load & Cache)

In [7]:
def load_raw_data():
    # 1. 원본 캐시 파일 확인 및 로드
    if Path(Config.OUTPUT_RAW_DATA_CACHE).exists():
        print(f"> Raw data cache found. Loading from: {Config.OUTPUT_RAW_DATA_CACHE}")
        with open(Config.OUTPUT_RAW_DATA_CACHE, 'rb') as f:
            df = pickle.load(f)
        print(f"> Successfully loaded {len(df)} raw records.")
        return df

    # 2. 원본 JSON 파일 로딩
    path = Config.DATA_FILE_PATH
    if not os.path.exists(path):
        raise FileNotFoundError(f"> Not found: {path}.")

    rows = []
    print(f"> Raw data cache not found. Loading raw JSON from {path}...")
    with open(path, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Loading JSON"):
            rows.append(json.loads(line))
    df = pd.DataFrame(rows)
    
    # 3. 로드된 원본 DataFrame 캐시 저장
    Path(os.path.dirname(Config.OUTPUT_RAW_DATA_CACHE)).mkdir(parents=True, exist_ok=True)
    with open(Config.OUTPUT_RAW_DATA_CACHE, 'wb') as f:
        pickle.dump(df, f)
    
    print(f"\n> Raw data loaded and cached. Total records: {len(df)}")
    return df

In [8]:
df_raw = load_raw_data()

> Raw data cache found. Loading from: ./dataset/arxiv_raw_cache.pkl
> Successfully loaded 2860945 raw records.


## 1.4. 전처리, 샘플링 및 최종 캐시 생성 (Preprocessing, Sampling & Final Cache)

In [9]:
def preprocess_data(df):
    """연도 추출, 필터링, 텍스트 병합을 수행하고 중간 결과를 저장합니다."""
    
    # 1. 중간 캐시 파일 확인 및 로드 (Cache Check)
    if Path(Config.OUTPUT_PROCESSED_DATA_INTERIM).exists():
        print(f"> Intermediate processed data found. Loading from: {Config.OUTPUT_PROCESSED_DATA_INTERIM}")
        with open(Config.OUTPUT_PROCESSED_DATA_INTERIM, 'rb') as f:
            df_processed = pickle.load(f)
        print(f"> Successfully loaded {len(df_processed)} pre-processed records. Skipping preprocessing.")
        return df_processed
        
    print("> Intermediate cache not found. Starting preprocessing...")
    df_temp = df.copy()

    # 2. 연도 추출 
    df_temp['year'] = df_temp['versions'].apply(extract_year_from_versions)
    df_temp['year'] = df_temp['year'].astype('Int64', errors='ignore')

    # 3. 필터링 및 텍스트 병합
    df_processed = df_temp.dropna(subset=['title', 'abstract', 'year']).reset_index(drop=True)
    df_processed['text'] = df_processed['title'].astype(str) + " " + df_processed['abstract'].astype(str)
    df_processed = df_processed[df_processed['text'].str.strip().str.len() > 10].reset_index(drop=True) 
    print(f"> After filtering (NaN, short text): {len(df_processed)} records.")
    
    # 4. 중간 결과 캐시 저장
    Path(os.path.dirname(Config.OUTPUT_PROCESSED_DATA_INTERIM)).mkdir(parents=True, exist_ok=True)
    with open(Config.OUTPUT_PROCESSED_DATA_INTERIM, 'wb') as f:
        pickle.dump(df_processed, f)
        
    print(f"\n> Preprocessing Done. Intermediate Total Records: {len(df_processed)}")
    return df_processed

In [10]:
df_processed = preprocess_data(df_raw)

> Intermediate processed data found. Loading from: ./dataset/processed_arxiv_interim.pkl
> Successfully loaded 2860911 pre-processed records. Skipping preprocessing.


In [ ]:
# ==============================================================================
# 5. 샘플링 및 최종 캐시 생성 (Sampling & Final Cache)
# ==============================================================================

def apply_sampling_and_final_slot(df_processed):
    """전처리된 DataFrame을 연간 샘플링하고 최종 Time Slot을 생성합니다."""
    
    # 1. 최종 캐시 파일 확인 및 로드 (Cache Check)
    if Path(Config.OUTPUT_PROCESSED_DATA_PATH).exists():
        print(f"> Final sampled data found. Loading from: {Config.OUTPUT_PROCESSED_DATA_PATH}")
        with open(Config.OUTPUT_PROCESSED_DATA_PATH, 'rb') as f:
            df_final = pickle.load(f)
        print(f"> Successfully loaded {len(df_final)} final sampled records. Skipping sampling.")
        return df_final
        
    print("> Sampled cache not found. Starting annual stratified sampling...")

    # 2. 연도별 계층적 샘플링 (Stratified Sampling)
    df_sampling_base = df_processed.copy()
    if len(df_sampling_base) > 0:
        sample_size = Config.SAMPLES_PER_YEAR
        def safe_sample(group):
            if len(group) < sample_size:
                return group
            return group.sample(n=sample_size, random_state=42)
            
        df_sampled = df_sampling_base.groupby('year', group_keys=False).apply(safe_sample).reset_index(drop=True)
        print(f"> Stratified Sampling (by Year) Applied. Total Samples: {len(df_sampled)}")
        df_final = df_sampled
    else:
        df_final = df_processed
        
    # 3. Time Slot 생성 및 최종 필터링
    df_final['slot'] = df_final['year'].apply(to_5yr_slot)
    df_final = df_final.dropna(subset=['slot']).reset_index(drop=True) 

    # 4. 최종 결과 캐시 저장
    Path(os.path.dirname(Config.OUTPUT_PROCESSED_DATA_PATH)).mkdir(parents=True, exist_ok=True)
    with open(Config.OUTPUT_PROCESSED_DATA_PATH, 'wb') as f:
        pickle.dump(df_final, f)
        
    print(f"\n> Sampling Done. Final Total Records: {len(df_final)}")
    print(f"--- Final Slot Distribution ---\n{df_final['slot'].value_counts().sort_index()}")
    
    return df_final

In [14]:
df = apply_sampling_and_final_slot(df_processed)
texts = df['text'].tolist()

> Sampled cache not found. Starting annual stratified sampling...


/var/folders/tg/v_32vyl14g5dm3wm3452sjrc0000gn/T/ipykernel_31138/2928811203.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_sampling_base.groupby('year', group_keys=False).apply(safe_sample).reset_index(drop=True)


> Stratified Sampling (by Year) Applied. Total Samples: 3500

> Sampling Done. Final Total Records: 3500
--- Final Slot Distribution ---
slot
1991-1995    500
1996-2000    500
2001-2005    500
2006-2010    500
2011-2015    500
2016-2020    500
2021-2025    500
Name: count, dtype: int64


## 1.5. SBERT 모델 로드 및 인퍼런스 프로파일링 함수

In [19]:
def load_sbert(model_option='FP32'):
    """최적화 옵션에 따라 SBERT 모델을 로드합니다."""
    model = SentenceTransformer(Config.MODEL_NAME)
    if torch.backends.mps.is_available():
        device = "mps"
        os.environ["TOKENIZERS_PARALLELISM"] = "false"
    elif torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
        
    model.to(device)

    if model_option == 'FP16' and device in ("cuda", "mps"):
        try:
            model.half()
        except:
            print(f"> Warning: FP16 failed on {device}. Falling back to FP32.")
            model_option = 'FP32'
    
    if model_option == 'COMPILE':
        if hasattr(torch, 'compile'):
            model.encode = torch.compile(model.encode, mode="reduce-overhead")
            print("> torch.compile enabled (if available).")
        else:
            model_option = 'FP32'

    model.eval()
    return model, device, model_option


def profile_and_embed(texts, batch_size=Config.BATCH_SIZE):
    """최적화 옵션별 성능을 프로파일링하고 결과를 반환합니다."""
    
    profile_results = []
    options = ['FP32', 'FP16']
    profile_texts = texts[:Config.PROFILE_SAMPLE_SIZE]
    
    print("\n--- Inference Profiling Start (Optimization Demo) ---")
    for opt in options:
        try:
            # 1. 모델 로드 및 워밍업
            model, device, final_opt = load_sbert(model_option=opt)
            model.encode(profile_texts[:batch_size], batch_size=batch_size, device=device)
            
            # 2. 실제 프로파일링
            start_time = time.time()
            for i in range(0, len(profile_texts), batch_size):
                batch = profile_texts[i:i+batch_size]
                with torch.no_grad():
                    _ = model.encode(batch, batch_size=batch_size, device=device)
            end_time = time.time()
            
            duration = end_time - start_time
            throughput = len(profile_texts) / duration 
            mem = 0
            if device == 'cuda':
                mem = torch.cuda.max_memory_allocated(device=device) / (1024**3)
            elif device == 'mps':
                try:
                    mem = torch.mps.current_allocated_memory() / (1024**3)
                except AttributeError:
                    print(f"\n[Warning] Accurate memory tracking for MPS failed. Reporting 0.0 GB.")
                    mem = 0.0      
            profile_results.append({
                'Option': final_opt,
                'Throughput (s/s)': round(throughput, 2),
                'Memory (GB)': round(mem, 2),
                'Duration (s)': round(duration, 2)
            })
            
            print(f"[{final_opt}] Throughput: {throughput:.2f} s/s, Mem: {mem:.2f} GB")
            del model
            if device in ('cuda', 'mps'):
                torch.cuda.empty_cache()
                
        except Exception as e:
            print(f"Profiling failed for [{opt}]: {e}")
            
    profile_df = pd.DataFrame(profile_results)
    print("\n--- Profiling Result Table  ---")
    print(profile_df)
    
    return profile_df

In [20]:
profile_df = profile_and_embed(texts, batch_size=Config.BATCH_SIZE)


--- Inference Profiling Start (Optimization Demo) ---
[FP32] Throughput: 34.27 s/s, Mem: 1.90 GB
[FP16] Throughput: 40.58 s/s, Mem: 2.69 GB

--- Profiling Result Table  ---
  Option  Throughput (s/s)  Memory (GB)  Duration (s)
0   FP32             34.27         1.90         58.36
1   FP16             40.58         2.69         49.29


## 1.6 전체 데이터셋 임베딩 생성 및 저장

In [ ]:
def generate_final_embedding(texts, profile_df, batch_size=Config.BATCH_SIZE):
    """전체 데이터셋에 대해 임베딩을 생성하고 결과를 저장합니다."""
    
    print("\n--- Final Embedding Generation (FP16 Selected for Stability) ---")
    
    model_final, device_final, final_opt = load_sbert(model_option='FP16')
    print(f"> Final embedding using: {final_opt} on {device_final}")
    
    final_embs = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch = texts[i:i+batch_size]
        with torch.no_grad():
            out = model_final.encode(
                batch,
                batch_size=batch_size,
                convert_to_tensor=True,
                device=device_final
            )
        final_embs.append(out.cpu())
    
    embeddings = torch.cat(final_embs, dim=0)

    # 결과 캐시
    Path(os.path.dirname(Config.OUTPUT_EMBEDDING_PATH)).mkdir(parents=True, exist_ok=True)
    with open(Config.OUTPUT_EMBEDDING_PATH, 'wb') as f:
        pickle.dump({
            "ids": df['id'].tolist(),
            "emb": embeddings,
            "slot": df['slot'].tolist(),
            "year": df['year'].tolist(),
            "profile": profile_df.to_dict('records') # 최적화 결과 저장
        }, f)

    print(f"\n> Done. Final Embeddings ({embeddings.shape}) and profiling results saved to {Config.OUTPUT_EMBEDDING_PATH}")
    return embeddings

In [22]:
embeddings = generate_final_embedding(texts, profile_df, batch_size=Config.BATCH_SIZE)


--- Final Embedding Generation (FP16 Selected for Stability) ---
> Final embedding using: FP16 on mps


Embedding: 100%|██████████| 110/110 [01:30<00:00,  1.22it/s]


> Done. Final Embeddings (torch.Size([3500, 768])) and profiling results saved to ./dataset/arxiv_embeddings_fp16.pkl
